In [1]:
import networkx as nx
import pandas as pd
import json

In [2]:
# graphs = ["stexpanded", "memoryalpha", "swtor", "swg", "memorybeta", "marvel", "mcu", "starwars"]
graph = "stexpanded"
pair = "memoryalpha"
community_algorithm = "bigclam"

### Közösségkeresés, bennük fokszámeloszlás

In [ ]:
print(pair, "---", graph)
print("##################################################")

graph_file = "data/filtered_triples_weighted/" + graph + ".triples"
pair_file = "data/filtered_triples_weighted/" + pair + ".triples"
communities_file = "./results/communities_" + community_algorithm + "/" + graph + "_10000.txt"
mappings_file = "./data/triples_v2/" + graph + "_mapping.json"
exact_match_file = "./data/exact_match/" + pair + "-" + graph + "-formatted.json"
community_transform_file = "./results/community_transform/" + pair + "-" + graph + ".txt"

graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_graph = nx.Graph()
for row in graph_df.itertuples():
    G_graph.add_edge(int(row[1]), int(row[2]))

pair_df = pd.read_csv(pair_file, sep="###", engine="python", header=None)
G_pair = nx.Graph()
for row in pair_df.itertuples():
    G_pair.add_edge(int(row[1]), int(row[2]))

communities = []
with open(communities_file) as cf:
    for line in cf:
        numbers_set = {int(num) for num in line.strip().split("\t")}
        communities.append(numbers_set)

with open(mappings_file) as mf:
    mappings = json.load(mf)

mappings_dict = {value: key for key, value in mappings.items()}

with open(exact_match_file) as file:
    exact_match = json.load(file)

graph_multiplier = len(G_pair.nodes) / len(G_graph.nodes)
print(graph_multiplier)

extension_threshold = 0.10
size_threshold = 10

with open(community_transform_file, "w") as output:
    for community in communities:
        print("community size:", len(community))
        count_exact_match = 0
        node_set = set()
        for node in community:
            for em in exact_match:
                if em[1] == node:
                    count_exact_match += 1
                    node_set.add(em[0])
                    # print("match found:", node, "---", em[0])
        # print("exact match count:", count_exact_match)
        # print("node set:", node_set)

        required_neighbors = len(node_set) * extension_threshold
        result_nodes = []
        for node in G_pair.nodes:
            if node in node_set:
                continue
            neighbors = set(G_pair.neighbors(node))
            common_neighbors = neighbors & node_set
            if len(common_neighbors) >= required_neighbors:
                result_nodes.append(node)

        merged_nodes = node_set | set(result_nodes)
        # print("pair community:", merged_nodes)
        print("pair community size:", len(merged_nodes))
        print("exact matches:", count_exact_match)
        if len(community) * size_threshold >= len(merged_nodes):
            output.write(str(merged_nodes) + "\n")
        else:
            output.write("#####\n")
        print("##################################################")

### Can find

In [32]:
gold_file = "./data/gold_pairs/" + pair + "-" + graph + "_formatted.txt"

gold_pairs = list()
with open(gold_file) as gpf:
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])
# print(len(gold_not_exact))

can_find = 0
for p in gold_not_exact:
    for community in communities:
        if p[1] in community:
            can_find += 1
            break

print("in communities:", can_find, "/", len(gold_not_exact), " --- ", can_find / len(gold_not_exact) * 100, "%")

can find: 121 / 148  ---  81.75675675675676 %


### Gold pairs in pair communities

In [33]:
exact_match_file = "./data/exact_match/" + pair + "-" + graph + "-formatted.json"
gold_file = "./data/gold_pairs/" + pair + "-" + graph + "_formatted.txt"
communities_file = "./results/communities_" + community_algorithm + "/" + graph + "_10000.txt"
community_transform_file = "./results/community_transform/" + pair + "-" + graph + "-20%.txt"

with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_pairs = list()
with open(gold_file) as gpf:
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

communities = list()
with open(communities_file, "r") as cf:
    for line in cf:
        numbers_set = {int(num) for num in line.strip().split("\t")}
        communities.append(numbers_set)

communities_transform = list()
with open(community_transform_file, "r") as ctf:
    for line in ctf:
        if line == "#####\n":
            numbers_set = set()
        else:
            numbers_set = {int(num) for num in line.strip().replace("{", "").replace("}", "").split(", ")}
        communities_transform.append(numbers_set)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])
# print(len(gold_not_exact))

count = 0
for p in gold_not_exact:
    for i in range(0, len(communities)):
        if p[1] in communities[i] and p[0] in communities_transform[i]:
            count += 1
            print(len(communities[i]), "---", len(communities_transform[i]))
            break

print("can find:", count, "/", len(gold_not_exact), " --- ", count / len(gold_not_exact) * 100, "%")

9 --- 6532
26 --- 7228
61 --- 358
40 --- 328
11 --- 12228
13 --- 4991
16 --- 6793
11 --- 9594
15 --- 5705
10 --- 10233
26 --- 3244
15 --- 313
33 --- 449
42 --- 83
114 --- 252
10 --- 335
10 --- 265
22 --- 2309
21 --- 52
22 --- 2371
35 --- 2435
8 --- 4604
22 --- 2371
320 --- 209
16 --- 1823
11 --- 644
30 --- 1310
35 --- 3420
22 --- 2371
19 --- 884
40 --- 328
14 --- 61
13 --- 2584
72 --- 165
15 --- 9155
190 --- 131
72 --- 165
17 --- 824
13 --- 3010
42 --- 4042
20 --- 6663
13 --- 5467
3 --- 66406
33 --- 3389
94 --- 552
94 --- 552
94 --- 552
78 --- 66406
59 --- 194
23 --- 150
15 --- 3547
77 --- 225
23 --- 980
10 --- 1304
77 --- 225
26 --- 817
177 --- 4669
17 --- 3740
15 --- 4869
19 --- 2621
41 --- 3729
7 --- 11896
9 --- 3566
22 --- 2309
77 --- 225
12 --- 4664
98 --- 1998
13 --- 4808
8 --- 4604
15 --- 826
55 --- 367
54 --- 355
18 --- 3799
33 --- 1933
45 --- 546
18 --- 3799
55 --- 367
54 --- 355
41 --- 3729
55 --- 367
12 --- 4842
41 --- 3729
55 --- 367
23 --- 1667
77 --- 225
15 --- 6769
61 --